# Video Game Review Ratings Data

### Import Packages

In [217]:
import numpy as np
import pandas as pd
from io import BytesIO
import datetime as dt
from pandas import json_normalize
import json
import time
from igdb.wrapper import IGDBWrapper

### Pull Data

In [218]:
#Setup needed wrapper from IGDB database utilizing provided key to access database
wrapper = IGDBWrapper("zc2fgpfd3z73r4ryh18ibbowo4mr2x", "bakhlu7d3zakj1majlizb11lawdhuf")

In [226]:
#create a function to pull all of the records from igdb
def igdb_puller(table: str, query: str, query_limit: int):
    '''
    This function will take the table name and query as string arguments along with how many records to pull at once as
    an integer. It will pull the query limit amount of records each time until it pulls all of the records that meet 
    the query criteria. The function returns a dataframe of the requested data.
    '''
    
    offset = query_limit
    init_offset = query_limit
    
    byte_array = wrapper.api_request(table, query)
    df = json.loads(byte_array)
    df = json_normalize(df)
    df2 = df.copy()
    
    while len(df) != 0:
        byte_array = wrapper.api_request(table, query + 'offset ' + str(offset) + ';')
        df = json.loads(byte_array)
        df = json_normalize(df)
        df2 = pd.concat([df2, df])
        offset = offset + init_offset
        
    
    return df2

In [220]:
#create a table and query variable to pass into igdb_puller function
table = 'games'
query = 'fields id, name, first_release_date, total_rating, total_rating_count, category, platforms; limit 500;where  first_release_date != null & total_rating != null & total_rating_count > 10; sort total_rating desc;'

In [245]:
#call the igdb_puller function and save it into a dataframe
df = igdb_puller(table, query, 500)
df

In [247]:
#change the release date to pandas datetime format from the seconds format used by the igdb database
df['first_release_date'] = pd.to_datetime(df['first_release_date'], unit='s')

#create a new column to pull out the year attribute of the date for plotting later
df['release_year'] = df['first_release_date'].dt.year
df.head()

,id,category,first_release_date,name,platforms,total_rating,total_rating_count,release_year,release_month
0,51267,3,2009-09-15,Umineko: When They Cry Chiru,[6],99.540686,17,2009,9
1,45131,0,2013-09-17,Grand Theft Auto V: Special Edition,"[9, 12]",99.537825,18,2013,9
2,138851,12,2011-03-01,Not In The Groove,[6],99.033844,14,2011,3
3,14173,1,2003-12-31,Heroes of Might and Magic III: Horn of the Abyss,[6],98.995120,15,2003,12
4,74878,0,2017-11-07,Hitman: Game of the Year Edition,"[3, 6, 14, 48, 49]",98.832354,21,2017,11


In [248]:
#look at the numeric variables to get a sense of the data
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
id,7376.0,23922.209328,37486.101875,1.000000,2932.750000,8468.000000,21539.750000,205780.000000
category,7376.0,0.881779,2.531400,0.000000,0.000000,0.000000,0.000000,13.000000
total_rating,7376.0,74.465170,10.349515,10.133374,69.394631,75.717132,81.254442,99.540686
total_rating_count,7376.0,81.418791,192.305486,11.000000,16.000000,27.000000,63.000000,3700.000000
release_year,7376.0,2008.400759,9.278744,1971.000000,2003.000000,2011.000000,2016.000000,2022.000000
release_month,7376.0,7.262608,3.464460,1.000000,4.000000,8.000000,10.000000,12.000000


In [249]:
#look at the column data types
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7376 entries, 0 to 375
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id                  7376 non-null   int64         
 1   category            7376 non-null   int64         
 2   first_release_date  7376 non-null   datetime64[ns]
 3   name                7376 non-null   object        
 4   platforms           7376 non-null   object        
 5   total_rating        7376 non-null   float64       
 6   total_rating_count  7376 non-null   int64         
 7   release_year        7376 non-null   int64         
 8   release_month       7376 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(5), object(2)
memory usage: 576.2+ KB


In [250]:
#filter out the category column to only main game releases, this could have been done in the query as well
df = df[df['category'] == 0].copy()

#write the data to excel to import into Tableau for visualization
df.to_excel('video_game_scores.xlsx')